In [ ]:
import tensorflow as tf
from absl import app, flags, logging
from absl.flags import FLAGS
import yolov4.core.utils as utils
from yolov4.core.yolov4 import filter_boxes
from tensorflow.python.saved_model import tag_constants
from PIL import Image
import cv2
import numpy as np
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import coremltools as ct 
import time
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"


In [ ]:
flags.DEFINE_string('framework', 'tf', '(tf, tflite, trt')
flags.DEFINE_string('weights', 'yolov4/Pleural_Line_TensorFlow',
                    'path to weights file')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_boolean('tiny', False, 'yolo or yolo-tiny')
flags.DEFINE_string('model', 'yolov4', 'yolov3 or yolov4')
flags.DEFINE_string('image', '/shared_data/YOLO_Updated_PL_Model_Results/Sliding/image_677741729740_clean/frame0.png', 'path to input image')
flags.DEFINE_string('output', 'result.png', 'path to output image')
flags.DEFINE_float('iou', 0.45, 'iou threshold')
flags.DEFINE_float('score', 0.25, 'score threshold')

In [ ]:
FLAGS(['yolov4/detect.py'])

In [ ]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
STRIDES, ANCHORS, NUM_CLASS, XYSCALE = utils.load_config(FLAGS)
input_size = FLAGS.size
image_path = FLAGS.image

# original_image = cv2.imread("/shared_data/YOLO_Updated_PL_Model_Results/Sliding/image_677741729740_clean/frame0.png")
original_image = cv2.imread("/shared_data/YOLO_Updated_PL_Model_Results/No_Sliding/Image_262499828648_clean/frame0.png")
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# image_data = utils.image_preprocess(np.copy(original_image), [input_size, input_size])
image_data = cv2.resize(original_image, (input_size, input_size))
image_data = image_data / 255.
# image_data = image_data[np.newaxis, ...].astype(np.float32)

images_data = []
for i in range(1):
    images_data.append(image_data)
images_data = np.asarray(images_data).astype(np.float32)

if FLAGS.framework == 'tflite':
    interpreter = tf.lite.Interpreter(model_path=FLAGS.weights)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(input_details)
    print(output_details)
    interpreter.set_tensor(input_details[0]['index'], images_data)
    interpreter.invoke()
    pred = [interpreter.get_tensor(output_details[i]['index']) for i in range(len(output_details))]
    if FLAGS.model == 'yolov3' and FLAGS.tiny == True:
        boxes, pred_conf = filter_boxes(pred[1], pred[0], score_threshold=0.25, input_shape=tf.constant([input_size, input_size]))
    else:
        boxes, pred_conf = filter_boxes(pred[0], pred[1], score_threshold=0.25, input_shape=tf.constant([input_size, input_size]))
else:
    saved_model_loaded = tf.saved_model.load(FLAGS.weights, tags=[tag_constants.SERVING])
    infer = saved_model_loaded.signatures['serving_default']
    
    start_time = time.time()
   
    print(image_data.shape)
    print(image_data.min())
    print(image_data.max())
    batch_data = tf.constant(images_data)
    
    pred_bbox = infer(batch_data)
    print(pred_bbox)
    print(pred_bbox['tf_op_layer_concat_18'].shape)
    for key, value in pred_bbox.items():
        boxes = value[:, :, 0:4]
        pred_conf = value[:, :, 4:]
        print("VALUE", value)
        
    print('boxes')
    print(tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)))

    print('conf')
    print(tf.reshape(
        pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])))

boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
    boxes=tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)),
    scores=tf.reshape(
        pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])),
    max_output_size_per_class=50,
    max_total_size=50,
    iou_threshold=0.5,
    score_threshold=0.25
)
end_time = time.time()

pred_bbox = [boxes.numpy(), scores.numpy(), classes.numpy(), valid_detections.numpy()]
print(pred_bbox)
print(end_time-start_time)
# image = utils.draw_bbox(original_image, pred_bbox)
# # image = utils.draw_bbox(image_data*255, pred_bbox)
# image = Image.fromarray(image.astype(np.uint8))
# image.show()
# image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
# cv2.imwrite(FLAGS.output, image)

In [ ]:
# coreml_model = ct.convert(saved_model_loaded, source="tensorflow")
coreml_model = ct.convert("yolov4/Pleural_Line_TensorFlow/", source="tensorflow",
                          inputs=[ct.ImageType(name="input_1", shape=(1, 416, 416, 3), scale=1/255)],
                          outputs=[ct.TensorType(shape=(1, 18, 5))])
print(coreml_model)

In [52]:
import tensorflow as tf
import coremltools as ct
keras_model = tf.keras.models.load_model("yolov4/Pleural_Line_TensorFlow/", compile=False)
input_image = tf.keras.layers.Input(shape=(416, 416, 3), name="image")
pred_bbox = keras_model(input_image)
x = tf.keras.layers.MaxPooling2D(pool_size=(64, 64), strides=64)(input_image)
x = tf.keras.layers.Flatten()(x)
x1 = tf.keras.layers.Dense(1)(x)
x2 = tf.keras.layers.Dense(4)(x)

boxes = tf.reshape(pred_bbox[:, :, 0:4], (-1, 4))
scores = tf.reshape(pred_bbox[:, :, 4:], (-1, 1))

coordinates = tf.concat([boxes, x2], axis=0)
confidence = tf.concat([scores, x1], axis=0)

full_model = tf.keras.Model(inputs=[input_image], outputs=[coordinates, confidence])
# full_model.save('keras_model.tf')
full_model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 416, 416, 3  0           []                               
                                )]                                                                
                                                                                                  
 functional_1 (Functional)      (None, None, 5)      64003990    ['image[0][0]']                  
                                                                                                  
 max_pooling2d_9 (MaxPooling2D)  (None, 6, 6, 3)     0           ['image[0][0]']                  
                                                                                                  
 tf.__operators__.getitem_20 (S  (None, None, 4)     0           ['functional_1[0][0]']     

In [ ]:
keras_model.summary()

In [54]:
model = ct.convert(
    full_model, 
    inputs=[ct.ImageType(scale=1/255.0)]
)

output_sizes = [4, 1]
for i in range(2):
    ma_type = model._spec.description.output[i].type.multiArrayType
    ma_type.shapeRange.sizeRanges.add()
    ma_type.shapeRange.sizeRanges[0].lowerBound = 0
    ma_type.shapeRange.sizeRanges[0].upperBound = -1
    ma_type.shapeRange.sizeRanges.add()
    ma_type.shapeRange.sizeRanges[1].lowerBound = output_sizes[i]
    ma_type.shapeRange.sizeRanges[1].upperBound = output_sizes[i]
    del ma_type.shape[:]

2022-02-25 18:00:05.628994: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-25 18:00:05.904811: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-25 18:00:05.907324: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-25 18:00:05.909025: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-25 18:00:05.910679: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute 

In [ ]:
# ct.utils.rename_feature(model._spec, 'Identity', 'coordinates')
# ct.utils.rename_feature(model._spec, 'Identity_1', 'confidence')

# # Add descriptions to the inputs and outputs.
# model._spec.description.output[1].shortDescription = u"Boxes \xd7 Class confidence"
# model._spec.description.output[0].shortDescription = u"Boxes \xd7 [x, y, width, height] (relative to image size)"

# # Add metadata to the model.
# pipeline.spec.description.metadata.versionString = "YOLO v4 - Pleural Line Detector - 11/18/21"
# pipeline.spec.description.metadata.shortDescription = "YOLOv4 - Pleural Line Detector"
# pipeline.spec.description.metadata.author = "Converted to Core ML by Chris MacLellan"
# pipeline.spec.description.metadata.license = "None"

In [ ]:
model

In [ ]:
# model.save("YoloPleuralLine.mlmodel")

In [38]:
nms_spec = ct.proto.Model_pb2.Model()
nms_spec.specificationVersion = 5
for i in range(2):
    decoder_output = model._spec.description.output[i].SerializeToString()
    nms_spec.description.input.add()
    nms_spec.description.input[i].ParseFromString(decoder_output)
    nms_spec.description.output.add()
    nms_spec.description.output[i].ParseFromString(decoder_output)
    
nms_spec.description.output[0].name = "coordinates"
nms_spec.description.output[1].name = "confidence"

# output_sizes = [1, 4]
# for i in range(2):
#     ma_type = nms_spec.description.output[i].type.multiArrayType
#     ma_type.shapeRange.sizeRanges.add()
#     ma_type.shapeRange.sizeRanges[0].lowerBound = 0
#     ma_type.shapeRange.sizeRanges[0].upperBound = -1
#     ma_type.shapeRange.sizeRanges.add()
#     ma_type.shapeRange.sizeRanges[1].lowerBound = output_sizes[i]
#     ma_type.shapeRange.sizeRanges[1].upperBound = output_sizes[i]
#     del ma_type.shape[:]
    
#     ma_type2 = nms_spec.description.input[i].type.multiArrayType
#     ma_type2.shapeRange.sizeRanges.add()
#     ma_type2.shapeRange.sizeRanges[0].lowerBound = 0
#     ma_type2.shapeRange.sizeRanges[0].upperBound = -1
#     ma_type2.shapeRange.sizeRanges.add()
#     ma_type2.shapeRange.sizeRanges[1].lowerBound = output_sizes[i]
#     ma_type2.shapeRange.sizeRanges[1].upperBound = output_sizes[i]
#     del ma_type2.shape[:]
    
nms = nms_spec.nonMaximumSuppression
nms.confidenceInputFeatureName = "Identity_1"
nms.coordinatesInputFeatureName = "Identity"
nms.confidenceOutputFeatureName = "confidence"
nms.coordinatesOutputFeatureName = "coordinates"
nms.iouThresholdInputFeatureName = "iouThreshold"
nms.confidenceThresholdInputFeatureName = "confidenceThreshold"

# print(nms)

default_iou_threshold = 0.75
default_confidence_threshold = 0.25
nms.iouThreshold = default_iou_threshold
nms.confidenceThreshold = default_confidence_threshold
nms.pickTop.perClass = True

#print(type(nms))

labels = ['Pleural_Line']
nms.stringClassLabels.vector.extend(labels)

nms_model = ct.models.MLModel(nms_spec)

In [ ]:
nms_model

In [55]:
from coremltools.models import datatypes
from coremltools.models.pipeline import *
input_features = [ ("image", datatypes.Array(416, 416, 3))]
#                    ("iouThreshold", datatypes.Double()),
#                    ("confidenceThreshold", datatypes.Double()) ]

output_features = [ "confidence", "coordinates" ]

pipeline = Pipeline(input_features, output_features)

pipeline.add_model(model)
# pipeline.add_model(nms_model)

pipeline.spec.description.input[0].ParseFromString(
    model._spec.description.input[0].SerializeToString())
pipeline.spec.description.output[0].ParseFromString(
    model._spec.description.output[0].SerializeToString())
pipeline.spec.description.output[1].ParseFromString(
    model._spec.description.output[1].SerializeToString())

# Add descriptions to the inputs and outputs.
# pipeline.spec.description.input[1].shortDescription = "(optional) IOU Threshold override"
# pipeline.spec.description.input[2].shortDescription = "(optional) Confidence Threshold override"
pipeline.spec.description.output[1].shortDescription = u"Boxes \xd7 Class confidence"
pipeline.spec.description.output[0].shortDescription = u"Boxes \xd7 [x, y, width, height] (relative to image size)"

# Add metadata to the model.
pipeline.spec.description.metadata.versionString = "YOLO v4 - Pleural Line Detector - 11/15/21"
pipeline.spec.description.metadata.shortDescription = "YOLOv4 - Pleural Line Detector"
pipeline.spec.description.metadata.author = "Converted to Core ML by Chris MacLellan"
pipeline.spec.description.metadata.license = "None"

labels = ['Pleural_Line']
# Add the list of class labels and the default threshold values too.
user_defined_metadata = {
    # "iouThreshold": str(default_iou_threshold),
    # "confidenceThreshold": str(default_confidence_threshold),
    "classes": ",".join(labels)
}
pipeline.spec.description.metadata.userDefined.update(user_defined_metadata)


In [56]:
final_model = ct.models.MLModel(pipeline.spec)
final_model

input {
  name: "image"
  type {
    imageType {
      width: 416
      height: 416
      colorSpace: RGB
      imageSizeRange {
        widthRange {
          lowerBound: 416
          upperBound: 416
        }
        heightRange {
          lowerBound: 416
          upperBound: 416
        }
      }
    }
  }
}
output {
  name: "coordinates"
  shortDescription: "Boxes \303\227 [x, y, width, height] (relative to image size)"
  type {
    multiArrayType {
      dataType: FLOAT32
      shapeRange {
        sizeRanges {
          upperBound: -1
        }
        sizeRanges {
          lowerBound: 4
          upperBound: 4
        }
      }
    }
  }
}
output {
  name: "confidence"
  shortDescription: "Boxes \303\227 Class confidence"
  type {
    multiArrayType {
      dataType: FLOAT32
      shapeRange {
        sizeRanges {
          upperBound: -1
        }
        sizeRanges {
          lowerBound: 1
          upperBound: 1
        }
      }
    }
  }
}
metadata {
  shortDescription

In [57]:
final_model.save("YoloPleuralLine.mlmodel")
# Load a saved model
# loaded_model = ct.models.MLModel("../Models/yolo/yolov4-tiny-416.mlmodel")

In [ ]:
final_model

In [ ]:
from PIL import Image
import numpy as np
example_image = Image.open("/shared_data/YOLO_Updated_PL_Model_Results/Sliding/image_677741729740_clean/frame0.png").resize((416,416)).convert('RGB')
example_image

In [ ]:
out_dict = final_model.predict({"image": example_image, "iouThreshold": 0.5, "confidenceThreshold": 0.5})
out_dict